# Multi agent customer support system

In [16]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [17]:
from crewai import Agent, Task, Crew

In [18]:
import os
#from utils import get_openai_api_key

#openai_api_key = get_openai_api_key() 
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [75]:
from crewai import LLM

llm = LLM(
    model="openai/gpt-4o",        # <-- exact short name
    base_url="https://models.inference.ai.azure.com",
    api_key=os.getenv("GITHUB_TOKEN"),
    temperature=0.7,
    max_tokens=512,
)

## Role Playing, Focus and Cooperation

In [76]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
    llm=llm,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [77]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    #allow_delegation=False, cant delegate 
    llm=llm,
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

In [78]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [79]:
search_tool = SerperDevTool()

In [80]:
scrape_tool = ScrapeWebsiteTool()

In [81]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
    #website_url="https://niplan-market.onrender.com/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [82]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [83]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [84]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

## Running the Crew

Guardrails
By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [85]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0ae001ea-b183-4079-b70e-db171c77e830                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-08-05 16:21:30,966 - 15568 - rag_storage.py-rag_storage:134 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-08-05 16:21:34,796 - 15568 - rag_storage.py-rag_storage:134 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Provide a more concise and structured introduction that directly addresses the customer's inquiry            │
│  - Include more specific examples or case studies to illustrate the implementation of memory capabilities in    │
│  CrewAI                                                                                                         │
│  - Consider adding visual aids such as diagrams or charts to enhance the clarity of the explanation             │
│  - Include more specific examples to illustrate the points made in the response.                                │
│  - Provide additional links or resources for further reading on the topic of adding mem...                      │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 7168.34ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: DeepLearningAI just reached out with a super important ask:                                              │
│  I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you   │
│  provide guidance?                                                                                              │
│                                                                                                                 │
│  Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide     │
│  the best support possible.You must strive to provide a complete and accurate response to the customer's        │
│  inquiry.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Thought: Thought: I need to gather precise information on how to add memory to a Crew in CrewAI by reviewing   │
│  the content from the relevant documentation. This will ensure my response is accurate and comprehensive.       │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new

LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

Failed to add to long term memory: Failed to convert text into a Pydantic model due to error: 
litellm.RateLimitError: RateLimitError: OpenAIException - You exceeded your current quota, please check your plan 
and billing details. For more information on this error, read the docs: 
https://platform.openai.com/docs/guides/error-codes/api-errors.

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bf1f6825-4cfb-4528-9183-0aef25e2f793                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ae001ea-b183-4079-b70e-db171c77e830                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ---                                                                                              │
│                                                                                                                 │
│  Hi Andrew,                                                                                                     │
│                                                                                                                 │
│  Thank you so much for reaching out to us! I'm thrilled to assist you with setting up your Crew and adding      │
│  memory capabilities to your Crew in CrewAI. Memory is indeed a game-changer, allowing your AI agents to        │
│  retain and utilize contextual information, which makes their performance not just more intelligent but also    │
│  more adaptive to complex tasks. Let’s dive right in!                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **How to Add Memory to Your Crew in CrewAI**                                                               │
│                                                                                                                 │
│  Below, I’ve provided a detailed, step-by-step guide to integrate memory into your Crew. I’ve also included     │
│  some examples and resources for further clarity.                                                               │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **1. Understand the Role of Memory in CrewAI**                                                            │
│  Memory in CrewAI enables your AI agents to store, retrieve, and leverage relevant information throughout       │
│  their tasks. This is particularly useful for scenarios requiring historical context or continuity. Here are    │
│  some examples to help you understand its applications better:                                                  │
│  - **Conversational AI:** A chatbot can maintain a conversation history, enabling it to respond more naturally  │
│  and contextually.                                                                                              │
│  - **Research Assistance:** Your Crew can store and recall prior findings, references, or even intermediate     │
│  results for ongoing projects.                                                                                  │
│  - **Problem Solving:** When tackling complex, multi-step problems, memory allows your Crew to retain partial   │
│  results and build upon them efficiently.                                                                       │
│                                                       

In [74]:
from IPython.display import Markdown, display

clean = str(result).strip().removeprefix("```markdown").removesuffix("```").strip()
display(Markdown(clean))


Hello Andrew,

Thank you so much for reaching out with your important question! I’m thrilled to help guide you through setting up a Crew in CrewAI, and I’ll walk you through how to add memory capabilities so your agents can collaborate and retain context effectively. If you’re just getting started or looking to optimize your workflow, this guide should have you covered!

---

## 🚀 What is CrewAI?

CrewAI is a lightweight, high-performance Python framework designed from scratch (no dependencies on LangChain or other agent frameworks!) for building autonomous, collaborative AI teams—called “Crews.” Each Crew is made up of specialized AI agents, each with clear roles, tools, and goals, working together to tackle complex tasks.

**Key Features:**
- **Autonomous, role-based agents**
- **Seamless collaboration & workflow management**
- **Fast, modular, and easy to extend**

You can find more details in our [CrewAI documentation](https://docs.crewai.com/).

---

## 👥 How Do Crews and Agents Work?

Here’s a quick breakdown of CrewAI’s main components:

- **Crew:** The team manager—oversees agent collaboration, task assignments, and workflow.
- **Agent:** Each agent is a specialist (e.g., Researcher, Writer, Critic) with its own tools, memory, and objectives.
- **Process:** Defines how agents interact and collaborate on tasks.
- **Tasks:** The specific work items assigned to agents, with clear goals and expected outputs.

Agents can share insights, pass context, and build on each other’s work—just like a real team!

---

## 🧠 Adding Memory to Your Crew

Memory is key for multi-step reasoning, iterative refinement, and complex projects. In CrewAI, you can give your agents memory so they can remember previous interactions and use that context to improve their performance.

### Step-by-Step: Adding Memory to Agents in CrewAI

#### 1. **Install CrewAI**

If you haven’t already, you can install the latest version with:

```bash
pip install crewai
```

#### 2. **Define Your Agents and Specify Memory**

Each agent can be instantiated with its own memory module. This can be CrewAI’s built-in memory or a custom backend if you have special requirements (like persistent or shared memory).

**Example: Setting Up a Researcher Agent with Memory**

```python
from crewai import Agent, Crew, Memory